# Question 27 part 1

In [2]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
data_file = '/FileStore/tables/Exam3.csv'
df = spark.read.load(data_file, format = "csv",sep=",", inferSchema="true", header="true")

In [4]:
df_new = df.na.drop()

In [5]:
df_new.groupBy('Gender').count().show()

+------+-----+
Gender|count|
+------+-----+
Female| 202|
 Male| 193|
+------+-----+

# Part 2 Show reqression equation with coeffeicitnts and intercept

In [7]:
import pyspark.sql.functions as f
from pyspark.sql.types import *

In [8]:
from pyspark.ml.feature import OneHotEncoder,StringIndexer, VectorAssembler


Out[13]: DataFrame[summary: string, Married: string, Income: string, Balance: string, Gender_Num: string]

In [9]:

df_new.show(5)

+--------+-------+------+---+------+----------+-------+-------+-------+
RecordID| Income|Rating|Age|Gender|Gender_Num|Student|Married|Balance|
+--------+-------+------+---+------+----------+-------+-------+-------+
 1| 14.891| 283| 34| Male| 1| No| Yes| 333|
 2|106.025| 483| 82|Female| 0| Yes| Yes| 903|
 3|104.593| 514| 71| Male| 1| No| No| 580|
 4|148.924| 681| 36|Female| 0| No| No| 964|
 5| 55.882| 357| 68| Male| 1| No| Yes| 331|
+--------+-------+------+---+------+----------+-------+-------+-------+
only showing top 5 rows

In [10]:
student_indexer = StringIndexer(inputCol = "Married", outputCol="Married_index").fit(df_new)

In [11]:
df1 = student_indexer.transform(df_new)

In [12]:
Married_encoder = OneHotEncoder(inputCol="Married_index", outputCol="Married_vec")

In [13]:
df2 = Married_encoder.transform(df1)

In [14]:
df6 = VectorAssembler(inputCols=['Income', 'Balance', 'Married_index','Married_vec'], outputCol="features")


In [15]:
df_7 = df6.transform(df2)

In [16]:
df8 = df_7.select(['features', 'Gender_Num'])

In [17]:
from pyspark.ml.classification import LogisticRegression

In [18]:
df_renamed =df8.withColumnRenamed("Gender_Num", "label") # rename default_index to label

In [19]:
model_df = df_renamed

In [20]:
training_df,test_df=model_df.randomSplit([0.75,0.25])

In [21]:
log_reg=LogisticRegression().fit(training_df)

In [22]:
lr_summary=log_reg.summary

In [23]:
log_reg.intercept


Out[47]: -0.013217839573474386

In [24]:
log_reg.coefficients

Out[48]: DenseVector([0.0005, -0.0005, 0.1048, 0.1618])

$$p(genderNum) = \frac{e^{-0.013217839573474386 - 0.0005\*income -0.0005\*balance + 0.1048\*marriedvector= yes -.0318\*marriedvector=no}}{1+e^{{-0.013217839573474386 -0.0005\*income -0.1618\*balance + 0.1048\*marriedvector= yes -0.1618\*marriedvector=no}}}$$

# Q3 Describe model performance recall percision and AUC

In [27]:
print("Accuracy:",lr_summary.accuracy)
print("AUC:",lr_summary.areaUnderROC)
print("Precision:",lr_summary.precisionByLabel)
print("Recall:",lr_summary.recallByLabel)
Accuracy: 0.5292207792207793
AUC: 0.5378158349854474
Precision: [0.5319148936170213, 0.525]
Recall: [0.6369426751592356, 0.41721854304635764]

Accuracy: 0.5331010452961672
AUC: 0.5572415134714523
Precision: [0.546583850931677, 0.5158730158730159]
Recall: [0.5906040268456376, 0.47101449275362317]

The performance of this model is not good the accurancy is 53%. The auc is about 55% and that means it does not fit well. The recall for TP is about 63 percent and TN is 41%, this means that the model predicts correctly 59% of time for True postive and 47% fortrue negative. The precision is about 54 percent for False posigtiv e and about 51 percentg for false negative. 
This was not a great model